In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, f_oneway, kruskal, wilcoxon, mannwhitneyu
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import seaborn as sns

#наші дані
path = "student_habits_performance.csv"
df = pd.read_csv(path)

#40
if len(df) > 40:
    df = df.sample(n=40, random_state=42).reset_index(drop=True)
    print(f"Відібрано 40 випадкових записів.")

Відібрано 40 випадкових записів.


In [45]:
#Перевірка нормальності розподілу 
stat, p = shapiro(df['exam_score'])
print(f"\nТест={stat:.3f}, p-value={p:.3f}")
if p > 0.05:
    print("Розподіл нормальний(не відхиляємо H0).")
else:
    print("Розподіл не нормальний(відхиляємо H0).")



Тест=0.969, p-value=0.346
Розподіл нормальний(не відхиляємо H0).


In [46]:
#Однофакторний за Стьюдента(для нормальних даних)
if p > 0.05:
    #Однофакторний для gender
    gender_groups = [df[df['gender'] == g]['exam_score'] for g in df['gender'].unique()]
    f_val_gender, p_val_gender = f_oneway(*gender_groups)
    print(f"\nДля gender: F={f_val_gender:.2f}, p-value={p_val_gender:.3f}")

    #Однофакторний для part_time_job
    job_groups = [df[df['part_time_job'] == j]['exam_score'] for j in df['part_time_job'].unique()]
    f_val_job, p_val_job = f_oneway(*job_groups)
    print(f"Для part_time_job: F={f_val_job:.2f}, p-value={p_val_job:.3f}")



Для gender: F=0.03, p-value=0.868
Для part_time_job: F=1.10, p-value=0.302


In [47]:
#Непараметричний тест Крускала-Валліса для gender
if df['gender'].nunique() > 1:
    gender_groups = [df[df['gender'] == g]['exam_score'] for g in df['gender'].unique()]
    stat_gender, p_gender = kruskal(*gender_groups)
    print(f"\nКрускал-Валліс для gender: H={stat_gender:.2f}, p-value={p_gender:.3f}")

#Непараметричний тест Крускала-Валліса для part_time_job
if df['part_time_job'].nunique() > 1:
    job_groups = [df[df['part_time_job'] == j]['exam_score'] for j in df['part_time_job'].unique()]
    stat_job, p_job = kruskal(*job_groups)
    print(f"Крускал-Валліс для part_time_job: H={stat_job:.2f}, p-value={p_job:.3f}")



Крускал-Валліс для gender: H=0.01, p-value=0.924
Крускал-Валліс для part_time_job: H=1.16, p-value=0.282


In [48]:
#Манна-Уітні для попарного аналізу
if df['gender'].nunique() == 2:
    g1, g2 = df['gender'].unique()
    group1 = df[df['gender'] == g1]['exam_score']
    group2 = df[df['gender'] == g2]['exam_score']
    u_stat, p_mw = mannwhitneyu(group1, group2)
    print(f"\nМанна-Уітні для gender: U={u_stat:.2f}, p-value={p_mw:.3f}")

if df['part_time_job'].nunique() == 2:
    j1, j2 = df['part_time_job'].unique()
    group1 = df[df['part_time_job'] == j1]['exam_score']
    group2 = df[df['part_time_job'] == j2]['exam_score']
    u_stat, p_mw = mannwhitneyu(group1, group2)
    print(f"Манна-Уітні для part_time_job: U={u_stat:.2f}, p-value={p_mw:.3f}")



Манна-Уітні для gender: U=199.00, p-value=0.935
Манна-Уітні для part_time_job: U=226.00, p-value=0.288


In [28]:
column_before = 'exam_score'  
if column_before in df_before.columns:
    # Генеруємо дані для після
    df_after = df_before[column_before] * np.random.randint(1, 6, size=len(df_before))
    df_paired = pd.DataFrame({
        'exam_score_before': df_before[column_before],
        'exam_score_after': df_after
    })

    # Видаляємо рядки з відсутніми значеннями
    df_paired = df_paired.dropna()
    # Виконуємо Вілкоксонів тест
    stat_w, p_w = wilcoxon(df_paired['exam_score_before'], df_paired['exam_score_after'])

    print(f"\nКритерій Вілкоксона:")
    print(f"Statistic={stat_w:.2f}, p-value={p_w:.3f}")

    if p_w < 0.05:
        print("Різниця між до і після статистично значуща.")
    else:
        print("Різниця між до і після не є статистично значущою.")

else:
    print("ERROR")



Критерій Вілкоксона:
Statistic=0.00, p-value=0.000
Різниця між до і після статистично значуща.


In [49]:
# Порівняння результатів
print("\nРезультати порівняння за параметричними та непараметричними тестами:")

# Параметричний тест(критерій Стьюдента)
if p > 0.05:
    print(f"Для gender: параметричний тест - p-value={p_val_gender:.3f}")
    print(f"Для part_time_job: параметричний тест - p-value={p_val_job:.3f}")

# Непараметричний тест(Крускал-Валліс)
print(f"Для gender: непараметричний тест Крускала-Валліса - p-value={p_gender:.3f}")
print(f"Для part_time_job: непараметричний тест Крускала-Валліса - p-value={p_job:.3f}")

# Тест Вілкоксона для парних вибірок
print(f"\nТест Вілкоксона (до/після) - p-value={p_w:.3f}")

# Висновки
if p_val_gender < 0.05 and p_gender < 0.05 and p_w < 0.05:
    print("\nВисновок:Обидва тести(параметричний, непараметричний та Вілкоксон) показали статистично значущу різницю.")
elif p_val_gender >= 0.05 and p_gender >= 0.05 and p_w >= 0.05:
    print("\nВисновок:Обидва тести не виявили статистично значущої різниці.")
elif p_val_gender < 0.05 and p_gender >= 0.05 and p_w < 0.05:
    print("\nВисновок:Параметричний тест та тест Вілкоксона показали значущу різницю, а непараметричний не виявив.")
elif p_val_gender >= 0.05 and p_gender < 0.05 and p_w < 0.05:
    print("\nВисновок:Непараметричний тест та тест Вілкоксона показали значущу різницю, а параметричний не виявив.")



Результати порівняння за параметричними та непараметричними тестами:
Для gender: параметричний тест - p-value=0.868
Для part_time_job: параметричний тест - p-value=0.302
Для gender: непараметричний тест Крускала-Валліса - p-value=0.924
Для part_time_job: непараметричний тест Крускала-Валліса - p-value=0.282

Тест Вілкоксона (до/після) - p-value=0.000


In [50]:
# Висновки для тесту Вілкоксона
if p_w < 0.05:
    print(f"\nВисновок:Тест Вілкоксона показав статистично значущу різницю між групами до і після.")
else:
    print(f"\nВисновок:Тест Вілкоксона не виявив статистично значущої різниці між групами до і після")



Висновок:Тест Вілкоксона показав статистично значущу різницю між групами до і після.


In [52]:
#Параметричний тест(критерій Стьюдента)
if p > 0.05:
    print(f"Для gender: параметричний тест - p-value={p_val_gender:.3f}")
    print(f"Для part_time_job: параметричний тес - p-value={p_val_job:.3f}")
#Непараметричний тест(Крускал-Валліс)
print(f"Для gender: непараметричний тест Крускала-Валліса - p-value={p_gender:.3f}")
print(f"Для part_time_job: непараметричний тест Крускала-Валліса - p-value={p_job:.3f}")

#Тест Вілкоксона для порівняння до і після
print(f"\nДля парних даних(до і після): тест Вілкоксона - p-value={p_w:.3f}")

#Висновки 
if p_val_gender < 0.05 and p_gender < 0.05:
    print("\nВисновок: Обидва(ANOVA і Крускал-Валліс) показали статистично значущу різницю за gender.")
elif p_val_gender >= 0.05 and p_gender >= 0.05:
    print("\nВисновок:Обидва тести не виявили статистично значущої різниці за gender.")
elif p_val_gender < 0.05 and p_gender >= 0.05:
    print("\nВисновок:Параметричний тест виявив значущу різницю, а непараметричний — ні.")
elif p_val_gender >= 0.05 and p_gender < 0.05:
    print("\nВисновок:Непараметричний тест виявив значущу різницю, а параметричний — ні.")

#Висновок 
if p_w < 0.05:
    print("Висновок:Тест Вілкоксона показав статистично значущу різницю між результатами до і після.")
else:
    print("Висновок:Тест Вілкоксона не показав статистично значущої різниці між результатами до і після.")


Для gender: параметричний тест - p-value=0.868
Для part_time_job: параметричний тес - p-value=0.302
Для gender: непараметричний тест Крускала-Валліса - p-value=0.924
Для part_time_job: непараметричний тест Крускала-Валліса - p-value=0.282

Для парних даних(до і після): тест Вілкоксона - p-value=0.000

Висновок:Обидва тести не виявили статистично значущої різниці за gender.
Висновок:Тест Вілкоксона показав статистично значущу різницю між результатами до і після.


<h1>Ми ознайомитись із групою непараметричних критеріїв: Вілкоксона, Манна-Уітні, Крускала-Валліса.<h1><h1>Незалежно від обраного критерію, суттєвого впливу gender або part_time_job на оцінки не виявлено.<h1>
<h1>Залежна група виявила різницю(адже вони помножені на 1-5 і дійсно різні)<h1>

